In [1]:
# !pip install openpyxl

In [11]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("/data/Augmented_data.csv")
df["Surname"] = df["Surname"].progress_apply(lambda x: str(x).capitalize())
df.head()

100%|██████████| 332397/332397 [00:00<00:00, 961469.52it/s]


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15658241,Olmstead,140.686089,Germany,Female,81,48,438091.980941,2,1,0,1.0,0,0,2,GOLD,2194.0,1-2-2023
1,15627162,Blesing,614.692750,Germany,Male,27,6,155245.230616,1,1,0,1.0,0,0,1,SILVER,675.0,1-3-2020
2,15986692,Leiva,665.305005,Germany,Male,35,7,157859.682813,2,1,1,1.0,0,0,5,SILVER,758.0,1-10-2020
3,15443069,Jamieson,541.210330,France,Male,36,7,0.000000,2,0,1,0.0,1,0,5,GOLD,1365.0,1-10-2020
4,15611786,Tsui,621.240000,Spain,Female,69,9,0.000000,1,0,1,0.0,0,0,5,PLATINUM,401.0,1-1-2020


In [12]:
cus_st = {}
for cus, st in zip(df["CustomerId"],df["Geography"]):
    if cus not in cus_st:
        cus_st[cus] = [st]
    else:
        if st not in cus_st[cus]:
            cus_st[cus].append(st)

In [13]:
df["Geography"] =  df["CustomerId"].progress_apply(lambda x: cus_st[x][0])

100%|██████████| 332397/332397 [00:00<00:00, 961979.68it/s]


In [14]:
cities = pd.read_excel("US_cities.xlsx")
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state_city[st].append(ct)
        
        
state_mapper = {}
for st in df["Geography"]:
    if st not in state_mapper:
        state_mapper[st] = np.random.choice(list(state_city.keys()))
        
city_mapper = {}
for cust_id, st in zip(df["CustomerId"],df["Geography"]):
    if cust_id not in city_mapper:
        city_mapper[cust_id] = np.random.choice(state_city[state_mapper[st]])

In [15]:
df["City"] = df["CustomerId"].progress_apply(lambda x: city_mapper[x])
df["month"] = df["date"].progress_apply(lambda x: x.split("-")[1])
df["year"] = df["date"].progress_apply(lambda x: x.split("-")[-1])
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: round(x,0))
df["Account_type"] = df["Tenure"].progress_apply(lambda x: "Gold" if x > 15 else "Premium" if x <= 15 and x >=8 else "Normal")

100%|██████████| 332397/332397 [00:00<00:00, 1229980.31it/s]


In [16]:
df["Other Accounts"] = df["Tenure"].progress_apply(lambda x: 1 if x > 7 else 0)

100%|██████████| 332397/332397 [00:00<00:00, 1066968.86it/s]


In [17]:
df["Salary Account"] = df["Satisfaction Score"].progress_apply(lambda x: 1 if x >= 3 else 0 )

100%|██████████| 332397/332397 [00:00<00:00, 1063985.36it/s]


In [18]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,...,Satisfaction Score,Card Type,Point Earned,date,City,month,year,Account_type,Other Accounts,Salary Account
0,15658241,Olmstead,141.0,Germany,Female,81,48,438091.980941,2,1,...,2,GOLD,2194.0,1-2-2023,Coppell,2,2023,Gold,1,0
1,15627162,Blesing,615.0,Germany,Male,27,6,155245.230616,1,1,...,1,SILVER,675.0,1-3-2020,Garland,3,2020,Normal,0,0
2,15986692,Leiva,665.0,Germany,Male,35,7,157859.682813,2,1,...,5,SILVER,758.0,1-10-2020,Laredo,10,2020,Normal,0,1
3,15443069,Jamieson,541.0,France,Male,36,7,0.000000,2,0,...,5,GOLD,1365.0,1-10-2020,Collierville,10,2020,Normal,0,1
4,15611786,Tsui,621.0,Spain,Female,69,9,0.000000,1,0,...,5,PLATINUM,401.0,1-1-2020,Washington,1,2020,Premium,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332392,15308570,Daigle,450.0,Germany,Female,43,14,313145.164928,1,1,...,5,PLATINUM,2220.0,1-1-2022,Grapevine,1,2022,Premium,1,1
332393,15864602,Griffith,316.0,France,Male,52,33,684971.208280,2,1,...,1,PLATINUM,2579.0,1-7-2022,Germantown,7,2022,Gold,1,0
332394,15626485,Lu,595.0,France,Female,26,8,85203.608400,1,1,...,1,SILVER,894.0,1-1-2020,Hendersonville,1,2020,Premium,1,0
332395,15984022,Battle,321.0,Germany,Male,53,30,425846.088483,1,1,...,5,DIAMOND,1962.0,1-5-2022,Tyler,5,2022,Gold,1,1


In [19]:
df.describe()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,Other Accounts,Salary Account
count,3.323970e+05,332397.000000,332397.000000,332397.000000,3.323970e+05,332397.000000,332397.000000,332397.000000,332397.000000,332397.000000,332397.000000,332397.000000,332397.000000,332397.000000,332397.000000
mean,1.553105e+07,359.972319,68.573203,35.813106,3.428347e+05,1.531810,0.700127,0.553621,3011.844463,0.135221,0.007987,2.948992,1819.284684,0.766301,0.583375
std,2.747476e+05,167.997593,34.399662,32.568198,4.662436e+05,0.512697,0.458203,0.497117,19794.229294,0.341960,0.089015,1.434245,1304.289545,0.423184,0.493000
min,1.500002e+07,56.000000,18.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,119.000000,0.000000,0.000000
25%,1.529730e+07,222.000000,40.000000,8.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,859.000000,1.000000,0.000000
50%,1.558321e+07,326.000000,59.000000,25.000000,1.582853e+05,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,3.000000,1414.000000,1.000000,1.000000
75%,1.575189e+07,482.000000,92.000000,58.000000,5.164369e+05,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.000000,2456.000000,1.000000,1.000000
max,1.599999e+07,850.000000,190.000000,116.000000,3.542322e+06,4.000000,1.000000,1.000000,199992.480000,1.000000,1.000000,5.000000,10698.000000,1.000000,1.000000


In [204]:
df.to_csv("/data/bank_customer_churn_data.csv", index=False)